In [1]:
# Import the necessary libraries

import pandas as pd

# Find path to PySpark.
import findspark
findspark.init("/Users/ugalugalu/spark-3.3.1-bin-hadoop3/")
import pyspark
from pyspark.sql import SQLContext

#Create a spark context object
sc = pyspark.SparkContext()



23/02/09 23:36:29 WARN Utils: Your hostname, Ubos-iMac.local resolves to a loopback address: 127.0.0.1; using 192.168.100.11 instead (on interface en0)
23/02/09 23:36:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/09 23:36:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1.Data Importation and Analysis

In [2]:
#Create Sqlcontext Object

sqlCtx = SQLContext(sc)

#Read in the data using the SQL context object
data = sqlCtx.read.csv('saf_stock.csv',header= True, inferSchema= True)
data.columns

/Users/ugalugalu/spark-3.3.1-bin-hadoop3/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

Observations:The Column names are Date,Open,High,Low,Close,Volume, and Adj Close

In [3]:
#Infering the schema from the data
data.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



Observations about the schema:All the six columns are of string columns

In [4]:
# Showing the first five columns
data.show(5)



+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [5]:
# Checking for Statistical Distribution
data.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

### 2.Data Preparation

In [6]:
# Import preparation modules
import pyspark.sql.functions as F
from pyspark.sql.functions import lit,when,col,expr,round

# Create a new data frame and round off the columns to two decimal places while adding the new HV column

data2=(
data.withColumn('HV',expr("High/Volume"))#Create the new column HV which is a ratio if High to Volume ratio of stocks traded\
    .withColumn('Open', F.format_number('Open', 2))# Round the Open Column to two decimal places\
    .withColumn('High', F.format_number('High', 2))# Round the high column to two decimal places\
    .withColumn('Low', F.format_number('Low', 2))# Round the Low column to two decimal places\
    .withColumn('Close', F.format_number('Close', 2))# Round the close column to two decimal places\
    .withColumn('Volume', round('Volume', 2))#round the volume column to two decimal places.Round function used in this case to solve comma issues\
    .withColumn('Adj Close', F.format_number('Adj Close', 2))# Round the close column to two decimal places\
    .withColumn('HV',F.format_number('HV', 10)))# Round the new HV column to 10 decimal places
 
data2.show(5)


+-------------------+-----+-----+-----+-----+--------+---------+------------+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|          HV|
+-------------------+-----+-----+-----+-----+--------+---------+------------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|    52.62|0.0000048197|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|    52.08|0.0000062908|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200|    51.83|0.0000046694|
|2012-01-06 00:00:00|59.42|59.45|58.87|59.00| 8069400|    51.46|0.0000073673|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18| 6679300|    51.62|0.0000089156|
+-------------------+-----+-----+-----+-----+--------+---------+------------+
only showing top 5 rows



In [7]:
#Register a table in SQL
table = data2.registerTempTable("SAF")

/Users/ugalugalu/spark-3.3.1-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [8]:
#Confirm that SAF has been registered
tables = sqlCtx.tableNames()

print(tables)

['saf']


In [9]:
#Question 1: What day had the peak high in price?
q = "SELECT \
         Date,max(High) AS Peak_High_Price \
     FROM SAF GROUP BY Date \
    ORDER BY Peak_High_Price DESC LIMIT 1 "
sqlCtx.sql(q).show()

+-------------------+---------------+
|               Date|Peak_High_Price|
+-------------------+---------------+
|2015-01-13 00:00:00|          90.97|
+-------------------+---------------+





13th January 2015 had Peak High Price

In [10]:
#Question 2: What is the mean of the close month?
q = "\
SELECT\
    MEAN(Close) AS MEAN\
        FROM SAF"

sqlCtx.sql(q).show()

+-----------------+
|             MEAN|
+-----------------+
|72.38844992050863|
+-----------------+



The mean of the close month was 72.38

In [11]:
# What is the max and min of the volume column

q = "SELECT\
     MIN(Volume) Min_Volume,MAX(Volume) Max_Volume\
         FROM SAF\
 "

sqlCtx.sql(q).show()

+----------+----------+
|Min_Volume|Max_Volume|
+----------+----------+
|   2094900|  80898100|
+----------+----------+



The minimum Volume was 2094900 while Maximum value was 80898100

In [12]:
# How many days was the close lower than 60 dollars?
q = "SELECT\
    COUNT(Date)\
    FROM SAF\
    WHERE Close <= 60\
    "
sqlCtx.sql(q).show()

+-----------+
|count(Date)|
+-----------+
|        116|
+-----------+



116 days had a close lower than 60 dollars

In [13]:
#What percentage of the time was the High greater than 80 Dollars
# Total column entries were computed earlier. An improved querry of this is to use Common Table Expressions for this querry

q = "SELECT\
     ROUND((COUNT(High)/1258*100),2) Percentage_Greater_Than_80\
         FROM SAF\
         WHERE High >= 80\
            "

sqlCtx.sql(q).show()

+--------------------------+
|Percentage_Greater_Than_80|
+--------------------------+
|                      9.14|
+--------------------------+



9.14% had time greater than 80 Dollars

In [14]:
# What is the Pearson correlation between High and Volume

q = "SELECT ROUND(corr(High,Volume),2) Pearson_Correlation\
          FROM SAF"

sqlCtx.sql(q).show()

+-------------------+
|Pearson_Correlation|
+-------------------+
|              -0.34|
+-------------------+



The Pearson Correlation Between High and Volume is -0.338, meaning that they are negatively correlated

In [15]:
#What is the Max High per Year

q = "SELECT\
     EXTRACT(YEAR FROM Date) Year,\
     MAX(High) Max_High\
     FROM SAF\
     GROUP BY Year\
     ORDER BY Max_High DESC"
sqlCtx.sql(q).show()

+----+--------+
|Year|Max_High|
+----+--------+
|2015|   90.97|
|2014|   88.09|
|2013|   81.37|
|2012|   77.60|
|2016|   75.19|
+----+--------+



In [16]:
#What is the average Close for each Calendar Month?
q = "SELECT\
    EXTRACT(MONTH FROM Date) Month,\
    ROUND(AVG(Close),2) Avg_Close\
    FROM SAF\
    GROUP BY Month\
    ORDER BY Month ASC"

sqlCtx.sql(q).show()

+-----+---------+
|Month|Avg_Close|
+-----+---------+
|    1|    71.45|
|    2|    71.31|
|    3|    71.78|
|    4|    72.97|
|    5|    72.31|
|    6|     72.5|
|    7|    74.44|
|    8|    73.03|
|    9|    72.18|
|   10|    71.58|
|   11|    72.11|
|   12|    72.85|
+-----+---------+

